In [7]:
import warnings
warnings.filterwarnings('ignore')

In [9]:
import os
import pprint
import tempfile

from typing import Dict, Text

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

In [10]:
# Ratings data
ratings = tfds.load('movielens/100k-ratings', split='train')
# Features of all the available movies.
movies = tfds.load('movielens/100k-movies', split = 'train')

Dl Completed...: 0 url [00:00, ? url/s]
Dl Completed...: 100%|██████████| 1/1 [00:12<00:00, 12.06s/ url]

Dl Completed...: 100%|██████████| 1/1 [00:12<00:00, 12.40s/ url]


Dataset movielens downloaded and prepared to C:\Users\huynn5\tensorflow_datasets\movielens\100k-ratings\0.1.1. Subsequent calls will reuse this data.


Dl Completed...: 0 url [00:00, ? url/s]
Dl Completed...: 100%|██████████| 1/1 [00:00<00:00, 76.93 url/s] 
Extraction completed...: 0 file [00:00, ? file/s]
Dl Completed...: 100%|██████████| 1/1 [00:00<00:00, 50.00 url/s]
                                                                        

Dataset movielens downloaded and prepared to C:\Users\huynn5\tensorflow_datasets\movielens\100k-movies\0.1.1. Subsequent calls will reuse this data.


In [14]:
for x in ratings.take(1).as_numpy_iterator():
    pprint.pprint(x)

{'bucketized_user_age': 45.0,
 'movie_genres': array([7], dtype=int64),
 'movie_id': b'357',
 'movie_title': b"One Flew Over the Cuckoo's Nest (1975)",
 'raw_user_age': 46.0,
 'timestamp': 879024327,
 'user_gender': True,
 'user_id': b'138',
 'user_occupation_label': 4,
 'user_occupation_text': b'doctor',
 'user_rating': 4.0,
 'user_zip_code': b'53211'}


In [15]:
for x in movies.take(1).as_numpy_iterator():
    pprint.pprint(x)

{'movie_genres': array([4], dtype=int64),
 'movie_id': b'1681',
 'movie_title': b'You So Crazy (1994)'}


In [16]:
tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed = 42, reshuffle_each_iteration = False)
train = shuffled.take(80_000)
test = shuffled.take(80_000).take(20_000)

In [23]:
# Figure out unique user ids and movie titles present in the data
# Map the raw values of our categorical features to embedding vectors in our model
# Need a vocabulary that maps a raw feature value to an integer in a continguous range:
# Allow us to look up the corresponding embeddings in our embedding tables

movie_titles = movies.batch(1_000)
user_ids = ratings.batch(1_000_000).map(lambda x: x['user_id'])
unique_movie_titles = np.unique(np.concatenate(list(movie_titles)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))
unique_movie_titles[:10]

InvalidArgumentError: {{function_node __wrapped__IteratorGetNext_output_types_3_device_/job:localhost/replica:0/task:0/device:CPU:0}} Cannot batch tensors with different shapes in component 0. First element had shape [1] and element 1 had shape [2]. [Op:IteratorGetNext] name: 